In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, make_scorer
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
# Настройка визуализаций
%matplotlib inline

In [ ]:
df = pd.read_excel('data/data.xlsx', index_col='Unnamed: 0')

In [ ]:
# т. к. пустых значений не много, а заполнять их не представляется возможным - удаляем их
df.dropna(how='any', inplace=True)

# Регрессия на SI

In [ ]:
# Из EDA мы помним о наличии больших выбросов. Уберем их
q_low, q_high = df["SI"].quantile([0.01, 0.99])
df_filtered = df[(df["SI"] >= q_low) & (df["SI"] <= q_high)]

# Логарифмирование SI (т.к. значения сильно скошены, помним из EDA)
df_filtered['SI_log'] = np.log1p(df_filtered['SI'])
y_si = df_filtered['SI_log']

X_si = df_filtered.drop(['IC50, mM', 'CC50, mM', 'SI', 'SI_log'], axis=1)

gbr_si = GradientBoostingRegressor(random_state=42)
selector_si = SelectFromModel(gbr_si, threshold="median")
selector_si.fit(X_si, y_si)

expert_features_si = [
    'MolLogP', 'TPSA', 'NumHDonors', 'NumHAcceptors', 
    'fr_halogen', 'qed', 'FractionCSP3', 'SPS'
]

selected_features_si = expert_features_si + list(X_si.columns[selector_si.get_support()])
selected_features_si = list(set(selected_features_si))

# Удаление мультиколлинеарных признаков
corr_matrix = X_si[selected_features_si].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
duplicates = [col for col in upper.columns if any(upper[col] > 0.9)]
final_features_si = [col for col in selected_features_si if col not in duplicates]

print(f"Итоговое число признаков: {len(final_features_si)}")

C:\Users\raven\AppData\Local\Temp\ipykernel_2848\2578173687.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['SI_log'] = np.log1p(df_filtered['SI'])  # Используем log1p для устойчивости


Итоговое число признаков: 87


In [ ]:
X_final_si = X_si[final_features_si]
X_train_si, X_test_si, y_train_si, y_test_si = train_test_split(
    X_final_si, y_si, test_size=0.2, random_state=42
)

param_grid = {
    'n_estimators': [50, 150, 200],
    'learning_rate': [0.1, 0.15],
    'max_depth': [5, 10],
}

gbr_si = GradientBoostingRegressor(random_state=42)
grid_search_gbr_si = GridSearchCV(
    gbr_si, 
    param_grid, 
    cv=5, 
    scoring='neg_mean_squared_error',
    n_jobs=-1)
grid_search_gbr_si.fit(X_train_si, y_train_si)

best_gbr_si = grid_search_gbr_si.best_estimator_
y_pred = best_gbr_si.predict(X_test_si)

print("\nМодель градиентного спуска:")
print(f"Параметры: {grid_search_gbr_si.best_params_}")
print(f"MAE на тесте: {mean_absolute_error(y_test_si, y_pred):.3f}")
print(f"R² на тесте: {r2_score(y_test_si, y_pred):.3f}")


Модель градиентного спуска:
Параметры: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50}
MAE на тесте: 0.815
R² на тесте: 0.332


In [ ]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 7],
    'learning_rate': [0.05, 0.1, 0.15],
    'gamma': [0, 0.1],
    'reg_alpha': [0, 0.1],
    'subsample': [0.8, 0.9], 
    'colsample_bytree': [0.8, 0.9]
}

xgb_model = XGBRegressor(random_state=42, objective='reg:squarederror')
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1
)

grid_search.fit(X_train_si, y_train_si)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_si)

print("\nМодель XGB:")
print("Лучшие параметры:", grid_search.best_params_)
print(f"MAE на тестовых данных: {mean_absolute_error(y_test_si, y_pred)}")
print(f"R2 на тестовых данных: {r2_score(y_test_si, y_pred)}")


Результаты для SI:
Лучшие параметры: {'colsample_bytree': 0.9, 'gamma': 0.1, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 50, 'reg_alpha': 0.1, 'subsample': 0.9}
MAE на тестовых данных: 0.8203366997961472
R2 на тестовых данных: 0.34268901586239153


C:\Users\raven\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Получились следующие результаты:
1) Модель XGB наилучшая среди остальных, хоть и менее устойчивая;
2) Модель GBR более устойчивая и почти равна прогностической силе XGB;
3) Модель ансамбля из моделей CC50 и IC50 оказалось несостоятельной.